# SQL Data Exploration 

For the purpose of keeping the SQL portion of this project available in our notebooks, we'll create an SQLite connection.

Our team at Pillow Palooza has exported the clean and preprocessed data and created a schema for us to use in our analysis. We have also included extra columns (from public data sources) to enhance the analysis. 

After meeting with your team, you wrote some questions to guide you along the analysis:
1. What are the most popular neighborhoods for short-term rentals in New York City?
2. What is the average rental price for short-term rentals in New York City, and how does it vary by neighborhood and property type?
3. What are the most commonly rented property types on Airbnb in New York City, and how does this vary by neighborhood?
4. What is the average length of stay for short-term rentals in New York City, and how does this vary by neighborhood and property type?
5. How has demand for short-term rentals in New York City changed over time, and are there any seasonal trends that could impact business decisions?

In [7]:
import pandas as pd
import sqlite3


In [8]:
# create the SQLite connection

cnn = sqlite3.connect('palooza.db')

In [9]:
# import the enriched data

prices = pd.read_csv('data_enriched/query_prices.csv')
room_types = pd.read_csv('data_enriched/query_room_types.csv')
reviews = pd.read_csv('data_enriched/query_reviews.csv')

In [10]:
# Save the data to SQLite

prices.to_sql('prices', cnn, if_exists='replace')
room_types.to_sql('room_types', cnn, if_exists='replace')
reviews.to_sql('reviews', cnn, if_exists='replace')



25209

In [12]:
# Load the ipython-sql extension
%load_ext sql

# Connect to the SQLite database
%sql sqlite:///palooza.db

'Connected: @palooza.db'

In [14]:
%%sql

SELECT COUNT(*)
FROM prices;


 * sqlite:///palooza.db
Done.


COUNT(*)
25202


NameError: name 'ue' is not defined